In [22]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
#website help from 
#https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

In [2]:
df = pd.read_csv('df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [33]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_ordinal_encoder(trial: Trial)-> OrdinalEncoder:
    params = {
        'handle_unknown': "use_encoded_value", 
        'unknown_value': -1
    }
    
    return OrdinalEncoder(**params)

def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': trial.suggest_categorical('drop', [None, 'first'])
    }
    
    return OneHotEncoder(**params)
    
Encoder = (
    OrdinalEncoder |
    OneHotEncoder 
    )

def instantiate_encoder (trial : Trial) -> Encoder:
    encoding_method = trial.suggest_categorical(
        'encoding_method', ['ordinal', 'onehot'])
    if encoding_method =='ordinal':
        encoder = instantiate_ordinal_encoder(trial)
    elif encoding_method =='onehot':
        encoder = instantiate_onehot_encoder(trial)
    
    return encoder

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_processor_ordinal(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_ordinal_encoder(trial)

    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

# Lasso

In [44]:
def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True)
    }

    return Lasso(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_lasso(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_lasso(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [45]:
from optuna import create_study

study_lasso = create_study(study_name='optimization', direction='maximize')

#study_lasso.optimize(lambda trial: objective_lasso(trial, X, y), n_trials=200)

[I 2024-07-11 09:50:20,023] A new study created in memory with name: optimization


In [11]:
print(study_lasso.best_value)
print(study_lasso.best_params)

0.9508124989217249
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00030733929099045147}


In [13]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_lasso.get_trials()[trial].number)
    score.append(study_lasso.get_trials()[trial].value)
    parameters.append(list(study_lasso.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_lasso = pd.DataFrame.from_dict(optuna_dict)
optuna_lasso

,Trial,Score,Parameters
0,0,0.946736,"[(scaling_method, maxabs), (encoding_method, o..."
1,1,0.932313,"[(scaling_method, maxabs), (encoding_method, o..."
2,2,0.941093,"[(scaling_method, robust), (encoding_method, o..."
3,3,0.948149,"[(scaling_method, maxabs), (encoding_method, o..."
4,4,0.947131,"[(scaling_method, robust), (encoding_method, o..."
...,...,...,...
195,195,0.950399,"[(scaling_method, minmax), (encoding_method, o..."
196,196,0.950790,"[(scaling_method, robust), (encoding_method, o..."
197,197,0.950787,"[(scaling_method, robust), (encoding_method, o..."
198,198,0.950707,"[(scaling_method, robust), (encoding_method, o..."


In [55]:
import plotly.express as px


fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# fig.write_image("images/optuna_lasso.png")

# Ridge

In [41]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 1, 100, log=True)
    }

    return Ridge(**params)


def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ridge(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ridge(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [43]:
from optuna import create_study

study_ridge = create_study(study_name='optimization', direction='maximize')

#study_ridge.optimize(lambda trial: objective_ridge(trial, X, y), n_trials=20)

[I 2024-07-11 09:50:20,011] A new study created in memory with name: optimization


In [17]:
print(study_ridge.best_value)
print(study_ridge.best_params)

0.9496764273611051
{'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'alpha': 24.817075515507273}


# EN

In [39]:
def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', .1, .7, log = True)
    }

    return ElasticNet(**params)



def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_en(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_en(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [40]:
from optuna import create_study

study_en = create_study(study_name='optimization', direction='maximize')

#study_en.optimize(lambda trial: objective_en(trial, X, y), n_trials=200)

[I 2024-07-11 09:50:19,990] A new study created in memory with name: optimization


In [66]:
# after adding in the L1 value
print(study_en.best_value)
print(study_en.best_params)

0.9508214083737423
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006921322276640134, 'l1_ratio': 0.4415555448483296}


In [29]:
print(study_en.best_value)
print(study_en.best_params)

0.950820633330468
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006178621404820522}


# XGB

In [18]:
def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 8),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 10, 20)
    }

    return XGBRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_xgb(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_xgb(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [64]:
from optuna import create_study

study_xgb = create_study(study_name='optimization', direction='maximize')

study_xgb.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=200)

[I 2024-07-11 10:29:44,482] A new study created in memory with name: optimization
[I 2024-07-11 10:29:51,126] Trial 0 finished with value: 0.8364908305073765 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.1080308948679474, 'loss': 'exponential', 'n_estimators': 136}. Best is trial 0 with value: 0.8364908305073765.
[I 2024-07-11 10:30:08,078] Trial 1 finished with value: 0.8565104045811929 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.0846428040972779, 'loss': 'linear', 'n_estimators': 370}. Best is trial 1 with value: 0.8565104045811929.
[I 2024-07-11 10:30:40,159] Trial 2 finished with value: 0.7952757615955968 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.005141603227491589, 'loss': 'square', 'n_estimators': 655}. Best is trial 1 with value: 0.8565104045811929.
[I 2024-07-11 10:30:48,019] Trial 3 finished with value: 0.7448139712853252 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.00017931592286634366, 'loss': '

[I 2024-07-11 10:40:01,252] Trial 33 finished with value: 0.8753590843058905 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.6024350272847602, 'loss': 'linear', 'n_estimators': 737}. Best is trial 26 with value: 0.8774689547481115.
[I 2024-07-11 10:40:21,171] Trial 34 finished with value: 0.8728928783696975 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.2654009501731239, 'loss': 'linear', 'n_estimators': 545}. Best is trial 26 with value: 0.8774689547481115.
[I 2024-07-11 10:40:52,654] Trial 35 finished with value: 0.7934918597318839 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.004933250216772752, 'loss': 'linear', 'n_estimators': 632}. Best is trial 26 with value: 0.8774689547481115.
[I 2024-07-11 10:41:24,786] Trial 36 finished with value: 0.8757836793814404 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.46203604853610064, 'loss': 'linear', 'n_estimators': 837}. Best is trial 26 with value: 0.8774689547481115.
[I 20

[I 2024-07-11 10:51:04,868] Trial 66 finished with value: 0.8758123761330122 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.9973979259829957, 'loss': 'exponential', 'n_estimators': 257}. Best is trial 26 with value: 0.8774689547481115.
[I 2024-07-11 10:51:09,165] Trial 67 finished with value: 0.8569649453947776 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.3535192217038593, 'loss': 'exponential', 'n_estimators': 97}. Best is trial 26 with value: 0.8774689547481115.
[I 2024-07-11 10:51:22,461] Trial 68 finished with value: 0.8716907649840246 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.66742528990224, 'loss': 'square', 'n_estimators': 421}. Best is trial 26 with value: 0.8774689547481115.
[I 2024-07-11 10:51:29,459] Trial 69 finished with value: 0.8711806391269619 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.45610533109317225, 'loss': 'exponential', 'n_estimators': 177}. Best is trial 26 with value: 0.8774689547481

[I 2024-07-11 10:59:47,274] Trial 99 finished with value: 0.8755423740862455 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.6824973622380154, 'loss': 'exponential', 'n_estimators': 542}. Best is trial 82 with value: 0.8775511844522542.
[I 2024-07-11 11:00:06,178] Trial 100 finished with value: 0.8741854500914303 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.3442583725470405, 'loss': 'exponential', 'n_estimators': 577}. Best is trial 82 with value: 0.8775511844522542.
[I 2024-07-11 11:00:22,301] Trial 101 finished with value: 0.8784307245353414 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.9823573285403469, 'loss': 'exponential', 'n_estimators': 551}. Best is trial 101 with value: 0.8784307245353414.
[I 2024-07-11 11:00:38,842] Trial 102 finished with value: 0.8766441784818184 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.7922773598535326, 'loss': 'exponential', 'n_estimators': 554}. Best is trial 101 with value: 0

[I 2024-07-11 11:11:34,443] Trial 132 finished with value: 0.8789121090590954 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8352590820357012, 'loss': 'exponential', 'n_estimators': 757}. Best is trial 132 with value: 0.8789121090590954.
[I 2024-07-11 11:11:56,772] Trial 133 finished with value: 0.8784909552130827 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8010041338257811, 'loss': 'exponential', 'n_estimators': 779}. Best is trial 132 with value: 0.8789121090590954.
[I 2024-07-11 11:12:19,278] Trial 134 finished with value: 0.8775812064170531 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.7387433824515962, 'loss': 'exponential', 'n_estimators': 777}. Best is trial 132 with value: 0.8789121090590954.
[I 2024-07-11 11:12:44,967] Trial 135 finished with value: 0.8767630733335983 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.6221177189781719, 'loss': 'exponential', 'n_estimators': 829}. Best is trial 132 with

[I 2024-07-11 11:24:21,982] Trial 164 finished with value: 0.877314124477112 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.6969213931778901, 'loss': 'exponential', 'n_estimators': 813}. Best is trial 132 with value: 0.8789121090590954.
[I 2024-07-11 11:24:43,855] Trial 165 finished with value: 0.8783884937280501 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9961532780615195, 'loss': 'exponential', 'n_estimators': 776}. Best is trial 132 with value: 0.8789121090590954.
[I 2024-07-11 11:25:05,211] Trial 166 finished with value: 0.8776096219390913 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8085737259790674, 'loss': 'exponential', 'n_estimators': 757}. Best is trial 132 with value: 0.8789121090590954.
[I 2024-07-11 11:25:26,740] Trial 167 finished with value: 0.8764420998120311 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.6410036971290285, 'loss': 'exponential', 'n_estimators': 733}. Best is trial 132 with 

[I 2024-07-11 11:36:23,321] Trial 196 finished with value: 0.8775997675038696 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8183212979242331, 'loss': 'exponential', 'n_estimators': 785}. Best is trial 181 with value: 0.8792409430087831.
[I 2024-07-11 11:36:44,409] Trial 197 finished with value: 0.8772858334424631 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.6852454265319394, 'loss': 'exponential', 'n_estimators': 723}. Best is trial 181 with value: 0.8792409430087831.
[I 2024-07-11 11:37:08,941] Trial 198 finished with value: 0.8790900930250967 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9979311630287945, 'loss': 'exponential', 'n_estimators': 891}. Best is trial 181 with value: 0.8792409430087831.
[I 2024-07-11 11:37:33,847] Trial 199 finished with value: 0.8759669000868548 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.5465691089452047, 'loss': 'exponential', 'n_estimators': 846}. Best is trial 181 with

In [28]:
print(study_xgb.best_value)
print(study_xgb.best_params)

0.951622985121114
{'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 979, 'learning_rate': 0.046054720840163293, 'max_depth': 4, 'subsample': 0.8282908794962028, 'colsample_bytree': 0.5510975070017484, 'min_child_weight': 10}


In [ ]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_lasso.get_trials()[trial].number)
    score.append(study_lasso.get_trials()[trial].value)
    parameters.append(list(study_lasso.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_lasso = pd.DataFrame.from_dict(optuna_dict)
optuna_lasso

In [ ]:
fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# RF

In [37]:
def instantiate_rf(trial : Trial) -> RandomForestRegressor:
    params = {
    'bootstrap':trial.suggest_categorical('bootstrap', [True]),
    'n_estimators': trial.suggest_int('n_estimators', 250, 500),
    'max_depth': trial.suggest_int('max_depth', 50, 75),
    'min_samples_split': trial.suggest_int('min_samples_split', 4, 6),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
    }
    
    return RandomForestRegressor(**params)
  
def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_rf(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_rf(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [38]:
from optuna import create_study

study_rf = create_study(study_name='optimization', direction='maximize')

#study_rf.optimize(lambda trial: objective_rf(trial, X, y), n_trials=5)

[I 2024-07-11 09:43:39,468] A new study created in memory with name: optimization
[I 2024-07-11 09:44:37,413] Trial 0 finished with value: 0.9083545094739573 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 253, 'max_depth': 60, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:45:24,724] Trial 1 finished with value: 0.8999706968816368 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 328, 'max_depth': 68, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:46:51,911] Trial 2 finished with value: 0.9081071301156255 and parameters: {'scaling_method': 'minmax', 'bootstrap': True, 'n_estimators': 427, 'max_depth': 73, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:48:22,636] Trial 3 finished with value: 0.9093564157184439 and parameters: {'sc

In [51]:
print(study_rf.best_value)
print(study_rf.best_params)

0.9106961840498735
{'scaling_method': 'robust', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 338, 'max_depth': 73, 'min_sample_split': 4, 'min_sample_leaf': 2}


# gbr

In [31]:
def instantiate_gbr(trial : Trial) -> GradientBoostingRegressor:
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'max_depth': trial.suggest_int ('max_depth', 2, 6),
    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    'min_samples_split': trial.suggest_int ('min_sample_split', 6, 10),
    'min_samples_leaf': trial.suggest_int ('min_sample_leaf', 1, 10)
    }
    
    return GradientBoostingRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_gbr(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_gbr(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [34]:
from optuna import create_study

study_gbr = create_study(study_name='optimization', direction='maximize')

study_gbr.optimize(lambda trial: objective_gbr(trial, X, y), n_trials=5)

[I 2024-07-11 09:36:55,471] A new study created in memory with name: optimization
[I 2024-07-11 09:38:00,789] Trial 0 finished with value: 0.10676969273467858 and parameters: {'scaling_method': 'maxabs', 'n_estimators': 680, 'learning_rate': 0.00015291122193602163, 'max_depth': 6, 'subsample': 0.7453028457760895, 'min_sample_split': 10, 'min_sample_leaf': 9}. Best is trial 0 with value: 0.10676969273467858.
[I 2024-07-11 09:38:22,650] Trial 1 finished with value: 0.28100365000054095 and parameters: {'scaling_method': 'standard', 'n_estimators': 605, 'learning_rate': 0.0006146473911796216, 'max_depth': 2, 'subsample': 0.8742574967407413, 'min_sample_split': 8, 'min_sample_leaf': 2}. Best is trial 1 with value: 0.28100365000054095.
[I 2024-07-11 09:39:24,934] Trial 2 finished with value: 0.2466584750875623 and parameters: {'scaling_method': 'standard', 'n_estimators': 533, 'learning_rate': 0.00043180685670930226, 'max_depth': 6, 'subsample': 0.8638913349214837, 'min_sample_split': 8, 'mi

In [35]:
print(study_gbr.best_value)
print(study_gbr.best_params)

0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_leaf': 10}


# Adaboost

In [53]:
def instantiate_ada(trial : Trial) -> AdaBoostRegressor:
    params = {
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'loss': trial.suggest_categorical('loss',['linear', 'square', 'exponential']),
    'n_estimators': trial.suggest_int('n_estimators', 1, 1000)
    }
    
    return AdaBoostRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ada(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ada(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [54]:
from optuna import create_study

study_ada = create_study(study_name='optimization', direction='maximize')

study_ada.optimize(lambda trial: objective_ada(trial, X, y), n_trials=50)

[I 2024-07-11 10:02:52,147] A new study created in memory with name: optimization
[I 2024-07-11 10:02:52,518] Trial 0 finished with value: 0.7697034560200977 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.5591223026880627, 'loss': 'linear', 'n_estimators': 5}. Best is trial 0 with value: 0.7697034560200977.
[I 2024-07-11 10:03:10,819] Trial 1 finished with value: 0.8546529541652251 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.06846055794485856, 'loss': 'linear', 'n_estimators': 404}. Best is trial 1 with value: 0.8546529541652251.
[I 2024-07-11 10:03:29,877] Trial 2 finished with value: 0.875164826045092 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.5252293433374021, 'loss': 'exponential', 'n_estimators': 551}. Best is trial 2 with value: 0.875164826045092.
[I 2024-07-11 10:03:51,103] Trial 3 finished with value: 0.7894060614839619 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.006602759458806035, 'loss': 'square'

[I 2024-07-11 10:14:44,856] Trial 33 finished with value: 0.866576708984014 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.09929974514686613, 'loss': 'exponential', 'n_estimators': 581}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:14:58,969] Trial 34 finished with value: 0.8703748675462952 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.2245325690083747, 'loss': 'exponential', 'n_estimators': 368}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:15:17,983] Trial 35 finished with value: 0.876282939613254 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.5987616594370728, 'loss': 'exponential', 'n_estimators': 614}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:15:40,207] Trial 36 finished with value: 0.8102956663214451 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.013357794606257661, 'loss': 'exponential', 'n_estimators': 454}. Best is trial 10 with value: 0.878

# Loop through?

In [61]:
studies = [study_lasso, study_ridge, study_en, study_xgb, study_rf, study_gbr, study_ada]
objectives = [objective_lasso, 
              objective_ridge, 
              objective_en,
              objective_xgb,
              objective_rf,
              objective_gbr,
              objective_ada]

# for study, objective in zip(studies, objectives):
#     study.optimize(lambda trial: objective(trial, X, y), n_trials=2)
    

[<function __main__.objective_lasso(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_ridge(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_en(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_xgb(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: O

In [52]:
for study in studies:
    print (study.best_value)
    print (study.best_params)

0.8735606638813536
{'scaling_method': 'robust', 'learning_rate': 0.30065504792403025, 'loss': 'exponential', 'n_estimators': 428}
0.7980960053211354
{'scaling_method': 'robust', 'learning_rate': 0.006736329679908936, 'loss': 'exponential', 'n_estimators': 615}
0.8177296198440576
{'scaling_method': 'maxabs', 'learning_rate': 0.04083092875953665, 'loss': 'exponential', 'n_estimators': 196}
0.951622985121114
{'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 979, 'learning_rate': 0.046054720840163293, 'max_depth': 4, 'subsample': 0.8282908794962028, 'colsample_bytree': 0.5510975070017484, 'min_child_weight': 10}
0.9094102521730991
{'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 494, 'max_depth': 62, 'min_samples_split': 4, 'min_samples_leaf': 1}
0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_l